In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Imports
from data_processing.gpt_processing import (
    generate_messages, 
    set_model_settings,
    run_immediate_chat_process,
    get_last_batch_response,
)

from data_processing.gpt_processing.pdf_journal_process import (
    batch_section, batch_translate, validate_and_save_metadata, save_sectioning_data, save_translation_data
)

%aimport time
%aimport json
%aimport datetime
%aimport logging
from pathlib import Path
from types import SimpleNamespace
from math import floor
from datetime import datetime
import json


In [ ]:
# File paths
project_dir = Path("/Users/phapman/Desktop/tnh-scholar/")
data_dir = project_dir / "data_processing"
journal_dir = data_dir / "processed_journal_data"
journal_name = "phat-giao-viet-nam-1956-02"
working_dir = journal_dir / journal_name
input_xml = working_dir / f"TEST_full_cleaned_{journal_name}.xml"
translation_xml_path = working_dir / f"translation_{journal_name}.xml"
section_batch_jsonl = working_dir / "processing_batch_files" /"section_batch.jsonl"
translate_batch_jsonl = working_dir / "processing_batch_files" / "translation_batch.jsonl"
section_metadata_out = working_dir / "section_metadata.json"
raw_json_metadata_path = working_dir / "raw_metadata_response.txt"
logfile = data_dir / "gpt_processing" / "processing_info.log"


In [ ]:
# constants
MAX_TOKEN_LIMIT = 20000
MAX_BATCH_RETRIES = 20  # Number of retries
BATCH_RETRY_DELAY = 5  # seconds to wait before retry

In [ ]:
# Set up the logger
def setup_logger(log_file_path):
    """
    Configures the logger to write to a log file and the console.
    """
    # Remove existing handlers
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)

    logging.basicConfig(
        level=logging.DEBUG,
        format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",  # Include logger name
        handlers=[
            logging.FileHandler(log_file_path, encoding="utf-8"),
            logging.StreamHandler()  # Optional: to log to the console as well
        ]
    )

    # Suppress DEBUG/INFO logs for specific noisy modules
    modules_to_suppress = ["httpx", "httpcore", "urllib3", "openai"]
    for module in modules_to_suppress:
        logger = logging.getLogger(module)
        logger.setLevel(logging.WARNING)  # Suppress DEBUG and INFO logs

    
    return logging.getLogger(__name__)

In [ ]:
logger = setup_logger(logfile)

In [ ]:
model_settings_section = {
    "gpt-4o": {
        "max_tokens": 5000,
        "temperature": 0.25
    }
}

In [ ]:
system_message_section = """You are a highly skilled assistant processing a Vietnamese Buddhist journal scanned from OCR. 
Use the title: "Journal of Vietnamese Buddhism."
You will be determining the journal sections by page number. You will also generate metadata for the full text and each section. 
You will return this metadata in JSON format.

Instructions:
1. Analyze the text and divide it into sections based on logical breaks, such as headings, topic changes, or clear shifts in content.
2. Ensure every page is part of  a section, even if that section is titled "blank page" or "title page," for example.
3. For each section, provide:
   - The original title in Vietnamese (`section_title_vi`).
   - The translated title in English (`section_title_en`).
   - The author's name if it is available (`section_author`). 
   - A one-paragraph summary of the section in English (`section_summary`).
   - A list of keywords for the section that are related to its content, these can be proper names, specific concepts, or contextual information.
   - The section's start and end page numbers (`start_page` and `end_page`).
   - Use "null" for any data that is not available (such as author name) for the section.

4. Return the output as a JSON object with the following schema:
{
    "journal_summary": "A one-page summary of the whole journal in English.",
    "sections": [
        {
            "title_vi": "Original title in Vietnamese",
            "title_en": "Translated title in English",
            "author": "Name of the author of the section",
            "summary": "One-paragraph summary of the section in English",
            "keywords": "A list of keywords for the section",
            "start_page":  X,
            "end_page":  Y
        },
        ...
    ]
}

5.  Ensure the JSON is well-formed and adheres strictly to the provided schema."""

In [ ]:
model_settings_translate = {
    "gpt-4o": {
        "max_tokens": 5000,  # a default value, updated per batch
        "temperature": 0.75
    }
}

In [ ]:
system_message_translate = """You are the world's foremost translator of Zen Master Thich Nhat Hanh's Vietnamese writing into English, following the language style of the plumvillage.org website.
The text is based on an OCR scan of a journal you edited from 1956-1958. Use the title: "Journal of Vietnamese Buddhism" for the journal when it is referenced.
You will be translating a single section of the journal and will be provided with the section title in English. 
You want advanced students of Thay to understand the text in its larger historical context, in the context of Vietnamese Buddhism, and in the context of your own life.
Translate for the most meaningful, typical, and eloquent English interpretation that is simple, yet poetic. Translate literally, don't add any content. 
Notes on the text can be added in the <notes>.
Make corrections in the text only where necessary (for example if words are missing) to create logical flow. Note all corrections in the <translation-notes>. 
Do not change <pagebreak> tag postioning. Each translated page must match its original page source as pages will be studied side by side with the original Vietnamese.
Infer paragraphs and text structure from the text layout.
Add XML tags for clarity, using only the following tags: 

   <section> for major sections.
   <subsection> for subsections.
   <title> for main titles of sections and subsections. 
   <subtitle> for subtitles of sections and subsections. 
   <heading> for headings that do not mark titles or subtitles
   <p> for paragraphs.
   <br/> for linebreaks that add meaning such as in poems or other structures.
   <TOC> for tables of contents
   <author> for authors of sections or subsections
   <ol> <ul> <li> for lists
   <i> for italics. 
   <b> for bold.
   <notes>
   <translation-notes>

You may use <notes> at the end of the section for notes on historical, cultural, spiritual, or other interesting elements of the text.
You may add <translation-notes> at the end of the section as a commentary to summarize your translation choices. 
For <translation-notes>, you may include information on Sino-Vietnamese, complex, unusual, poetic, or other interesting terms, and significant corrections to the text. 
In the <translation-notes> include the original Vietnamese terms for reference.

IMPORTANT: All titles, XML sections, text, and terms should be translated. Do not however, translate names of people; leave names in Vietnamese with diacritics.
IMPORTANT: Return pure XML with no formatting marks such as xml or ```.
IMPORTANT: The returned XML should begin and end with <section> tags."""

In [ ]:
input_xml

In [ ]:
translation_xml_path

In [ ]:
# Step 1: Sectioning
set_model_settings(model_settings_section)
metadata_serial_json = batch_section(input_xml, section_batch_jsonl, system_message_section, journal_name)
metadata_path = save_sectioning_data(section_metadata_out, raw_json_metadata_path, metadata_serial_json, journal_name)  

In [ ]:
# Step 2: Translating
set_model_settings(model_settings_translate)
if metadata_path:
    translation_data = batch_translate(input_xml, translate_batch_jsonl, metadata_path, system_message_translate, journal_name)

In [ ]:
translation_data

In [ ]:
save_translation_data(Path("test.xml"), translation_data)

In [ ]:
result = get_last_batch_response()

In [ ]:
print(result[0])

In [ ]:
validate_and_save_metadata(section_metadata_out, result[0], journal_schema)

In [ ]:
# def batch_sectioning(input_xml_path, output_json_path, raw_output_path, journal_name, max_retries=MAX_BATCH_RETRIES, retry_delay=BATCH_RETRY_DELAY):
#     """
#     Splits the journal content into sections using GPT, with retries for both starting and completing the batch.
#     """
#     journal_pages = get_text_from_file(input_xml_path)

#     # Create GPT messages for sectioning
#     user_message_wrapper = lambda text: f"{text}"
#     messages = generate_messages(system_message_section, user_message_wrapper, [journal_pages])

#     # Create JSONL file for batch processing
#     jsonl_file = create_jsonl_file_for_batch(messages, section_batch_jsonl, json_mode=True)

#     for attempt in range(max_retries):
#         try:
#             # Try to start the batch
#             batch = start_batch(jsonl_file, description=f"Batch for sectioning journal: {journal_name} | input file: {input_xml_path}")
#             batch_id = batch.id
#             if not batch_id:
#                 raise RuntimeError("Batch started but no ID was returned.")

#             print(f"Batch for sectioning started successfully on attempt {attempt + 1}. ID: {batch_id}")

#             # Poll for batch completion
#             json_results = poll_batch_for_response(batch_id)
#             if json_results:
#                 break # exit retry loop
#             else:
#                 raise RuntimeError("Unknown error in polling for batch response.", exc_info=True)

#         except Exception as e:
#             print(f"Attempt {attempt + 1} failed: {e}. Retrying batch process in {retry_delay} seconds...")
#             time.sleep(retry_delay)
#     else:
#         logger.error("Failed to complete batch sectioning after maximum retries.")
#         raise RuntimeError("Error: Failed to complete batch sectioning after maximum retries.")

#     # save raw result
#     try:
#         write_text_to_file(raw_output_path, json_results, force=True)
#     except Exception as e:
#         logger.error(f"failed to write raw response file: {raw_output_path}")
#         raise

#     # If successful, try to validate and save metadata and exit loop
#     try:
#         valid = validate_and_save_metadata(output_json_path, json_results, journal_schema)
#     except Exception as e:
#         logger.error(f"Error occurred while validating and saving metadata for journal {journal_name}: '{output_json_path}' (batch ID: {batch_id}).", exc_info=True)
#         raise
    
#     if valid:
#         logger.info(f"Successfully processed {journal_name}: {input_xml_path} with batch: {batch_id} and saved metadata to {output_json_path} ")
#         return output_json_path
        
    


In [ ]:
# # Step 1: Sectioning
# def batch_sectioning(input_xml_path, output_xml_path):
#     """
#     Splits the journal content into sections using the GPT model.
#     Saves the sectioned content back to XML.
#     """
#     # Load the input XML
#     journal_pages = load_xml(input_xml_path)
#     pages_content = [page.text for page in journal_pages]

#     # Create GPT messages for sectioning
#     user_message_wrapper = lambda text: f"Divide this content into sections:\n{text}"
#     messages = generate_messages(system_message_section, user_message_wrapper, pages_content)

#     # Create JSONL file for batch processing
#     jsonl_file = create_jsonl_file_for_batch(messages, section_batch_jsonl)

#     # Start the batch
#     batch = start_batch(jsonl_file, description="Batch for sectioning journal")
#     batch_id = batch.get("id")
#     if not batch_id:
#         print("Error: Failed to start batch for sectioning.")
#         return None

#     print(f"Batch for sectioning started with ID: {batch_id}")

#     # Poll for batch completion
#     results = poll_batch_status(batch_id)
#     if not results:
#         print("Error: Failed to retrieve sectioning batch results.")
#         return None

#     # Save sectioned content back to XML
#     for i, section_content in enumerate(results):
#         journal_pages[i].text = section_content  # Replace original content with sectioned content

#     save_xml(journal_pages, output_xml_path)
#     print(f"Sectioned journal saved to {output_xml_path}")

In [ ]:
# import os
# import json
# from gpt_processing.gpt_interface import (
#     set_api_client, 
#     generate_messages, 
#     create_jsonl_file_for_batch, 
#     start_batch, 
#     get_batch_response
# )
# from data_processing.xml_processing import (
#     load_xml, 
#     save_xml, 
#     extract_sections_from_xml
# )

# # Initialize OpenAI client
# set_api_client()

# # File paths
# INPUT_XML = "input_journal.xml"
# SECTIONED_XML = "sectioned_journal.xml"
# TRANSLATED_XML = "translated_journal.xml"
# BATCH_SECTION_JSONL = "section_batch.jsonl"
# BATCH_TRANSLATE_JSONL = "translate_batch.jsonl"

# # System messages
# SYSTEM_MESSAGE_SECTION = """
# You are a helpful assistant. Divide the text into meaningful sections and add XML tags:
# <section> for major sections, <subsection> for subsections, <title> for titles, and <p> for paragraphs.
# """
# SYSTEM_MESSAGE_TRANSLATE = """
# You are Thich Nhat Hanh translating from Vietnamese to English. Provide meaningful translations with appropriate XML tags:
# <section>, <subsection>, <title>, <p>.
# """

# # Step 1: Sectioning
# def batch_sectioning(input_xml, output_xml):
#     # Load the input XML and extract pages or chunks
#     journal_pages = load_xml(input_xml)
#     pages_content = [page.text for page in journal_pages]  # Assuming .text contains the text of each page

#     # Create GPT messages for sectioning
#     user_message_wrapper = lambda text: f"Divide this content into sections:\n{text}"
#     messages = generate_messages(SYSTEM_MESSAGE_SECTION, user_message_wrapper, pages_content)

#     # Create JSONL file for batch processing
#     jsonl_file = create_jsonl_file_for_batch(messages, BATCH_SECTION_JSONL)

#     # Start batch
#     batch = start_batch(jsonl_file, description="Batch for sectioning journal")
#     batch_id = batch.get("id")
#     if not batch_id:
#         print("Error: Failed to start batch for sectioning.")
#         return None

#     print(f"Batch for sectioning started with ID: {batch_id}")

#     # Poll for batch completion and retrieve results
#     results = get_batch_response(batch_id)
#     if not results:
#         print("Error: Failed to retrieve sectioning batch results.")
#         return None

#     # Save the sectioned content back to XML
#     for i, section_content in enumerate(results):
#         journal_pages[i].text = section_content  # Replace original content with sectioned content

#     save_xml(journal_pages, output_xml)
#     print(f"Sectioned journal saved to {output_xml}")

# # Step 2: Translation
# def batch_translation(input_xml, output_xml):
#     # Load the sectioned XML and extract sections or chunks for translation
#     sections = extract_sections_from_xml(input_xml)

#     # Create GPT messages for translation
#     user_message_wrapper = lambda section: f"Translate this section:\n{section}"
#     messages = generate_messages(SYSTEM_MESSAGE_TRANSLATE, user_message_wrapper, sections)

#     # Create JSONL file for batch processing
#     jsonl_file = create_jsonl_file_for_batch(messages, BATCH_TRANSLATE_JSONL)

#     # Start batch
#     batch = start_batch(jsonl_file, description="Batch for translating journal")
#     batch_id = batch.get("id")
#     if not batch_id:
#         print("Error: Failed to start batch for translation.")
#         return None

#     print(f"Batch for translation started with ID: {batch_id}")

#     # Poll for batch completion and retrieve results
#     results = get_batch_response(batch_id)
#     if not results:
#         print("Error: Failed to retrieve translation batch results.")
#         return None

#     # Save the translated content back to XML
#     for i, translated_content in enumerate(results):
#         sections[i].text = translated_content  # Replace original content with translated content

#     save_xml(sections, output_xml)
#     print(f"Translated journal saved to {output_xml}")

# # Main process
# if __name__ == "__main__":
#     # Step 1: Sectioning
#     print("Starting batch sectioning...")
#     batch_sectioning(INPUT_XML, SECTIONED_XML)

#     # Step 2: Translation
#     print("Starting batch translation...")
#     batch_translation(SECTIONED_XML, TRANSLATED_XML)

In [ ]:
# # Function schema for function calling
# function_schemas = [
#     {
#         "name": "save_processed_metadata",
#         "description": "Save metadata for a processed vietnamese journal, including sections and summaries, that will later be translated",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "journal_summary": {"type": "string", "description": "A one-page summary of the journal in English."},
#                 "sections": {
#                     "type": "array",
#                     "items": {
#                         "type": "object",
#                         "properties": {
#                             "section_title_vi": {"type": "string", "description": "The original title of the section in Vietnamese."},
#                             "section_title_en": {"type": "string", "description": "The translated title of the section in English."},
#                             "section_summary": {"type": "string", "description": "A one paragraph summary of the section in English."},
#                             "page_range": {
#                                 "type": "array",
#                                 "items": {"type": "integer"},
#                                 "minItems": 2,
#                                 "maxItems": 2,
#                                 "description": "The start and end page numbers of the section."
#                             }
#                         },
#                         "required": ["section_title_en", "section_title_vi", "section_summary", "page_range"]
#                     }
#                 }
#             },
#             "required": ["journal_summary", "sections"]
#         }
#     }
# ]

In [ ]:
# Step 2: Translation
# def batch_translate(input_xml_path, metadata_path):
#     """
#     Translates the journal sections using the GPT model.
#     Saves the translated content back to XML.
#     """
#     # Load the sectioned XML
#     section_metadata = #load json data from metadata_path and deserialize

#     # use the function split_xml_to_pages to get sections for translation:
#     sections = split_xml_pages(...)

#     # Create GPT messages for translation
#     user_message_wrapper = lambda section: f"Translate this section:\n{section}"
#     messages = generate_messages(system_message_translate, user_message_wrapper, sections)

#     # convert the blocks below to a series of nested try blocks with multiple attempts as in batch_section():
#     # add appropriate logging to match batch_section():

#     # Create JSONL file for batch processing
#     jsonl_file = create_jsonl_file_for_batch(messages, translate_batch_jsonl)

#     # Start the batch
#     batch = start_batch(jsonl_file, description="Batch for translating journal")
#     batch_id = batch.get("id")
#     if not batch_id:
#         print("Error: Failed to start batch for translation.")
#         return None

#     print(f"Batch for translation started with ID: {batch_id}")

#     # Poll for batch completion
#     results = poll_batch_for_response(batch_id)
#     if not results:
#         print("Error: Failed to retrieve translation batch results.")
#         return None

#     # Save translated content back to XML
#     translated_sections = []
#     for i, translated_content in enumerate(results):
#         translated_sections.append(translated_content)  # Replace original content with translated content

#     save_pages_to_xml(translated_sections, translated_xml)
#     print(f"Translated journal saved to {translated_xml}")

In [ ]:
# # old system message

# system_message_section = """
# You are a highly skilled assistant processing a Vietnamese journal scanned from OCR. 
# You will be determining the journal sections by page number. You will also generate summaries for the full text and each section. 
# You will return this metadata in JSON format.

# Instructions:
# 1. Analyze the text and divide it into sections based on logical breaks, such as headings, topic changes, or clear shifts in content.
# 2. Ensure every page is part of  a section, even if that section is titled "blank page" or "title page," for example.
# 3. For each section, provide:
#    - The original title in Vietnamese (`section_title_vi`).
#    - The translated title in English (`section_title_en`).
#    - The author's name if it is available (`section_author`). 
#    - A one-paragraph summary of the section in English (`section_summary`).
#    - A list of keywords for the section that are related to its content, these can be proper names, specific concepts, or contextual information.
#    - The section's start and end page numbers (`start_page` and `end_page`).
#    - Use "null" for any data that is not available (such as author name) for the section.

# 4. Return the output as a JSON object with the following schema:
# {
#     "journal_summary": "A one-page summary of the whole journal in English.",
#     "sections": [
#         {
#             "section_title_vi": "Original title in Vietnamese",
#             "section_title_en": "Translated title in English",
#             "section_author": "Name of the author of the section",
#             "section_summary": "One-paragraph summary of the section in English",
#             "section_keywords": "A list of keywords for the section",
#             "start_page":  X,
#             "end_page":  Y
#         },
#         ...
#     ]
# }

# 5.  Ensure the JSON is well-formed and adheres strictly to the provided schema.
# """

In [ ]:
# # Step 2: Translation
# def batch_translate(input_xml_path, output_xml_path):
#     """
#     Translates the journal sections using the GPT model.
#     Saves the translated content back to XML.
#     """
#     # Load the sectioned XML
#     section_metadata = 

#     # Create GPT messages for translation
#     user_message_wrapper = lambda section: f"Translate this section:\n{section}"
#     messages = generate_messages(system_message_translate, user_message_wrapper, sections)

#     # Create JSONL file for batch processing
#     jsonl_file = create_jsonl_file_for_batch(messages, translate_batch_jsonl)

#     # Start the batch
#     batch = start_batch(jsonl_file, description="Batch for translating journal")
#     batch_id = batch.get("id")
#     if not batch_id:
#         print("Error: Failed to start batch for translation.")
#         return None

#     print(f"Batch for translation started with ID: {batch_id}")

#     # Poll for batch completion
#     results = poll_batch_status(batch_id)
#     if not results:
#         print("Error: Failed to retrieve translation batch results.")
#         return None

#     # Save translated content back to XML
#     for i, translated_content in enumerate(results):
#         sections[i].text = translated_content  # Replace original content with translated content

#     save_pages_to_xml(sections, output_xml_path)
#     print(f"Translated journal saved to {output_xml_path}")

In [ ]:
# # Step 2: Translation
# def batch_translate(input_xml_path, metadata_path, journal_name, xml_output_path, max_retries=MAX_BATCH_RETRIES, retry_delay=BATCH_RETRY_DELAY):
#     """
#     Translates the journal sections using the GPT model.
#     Saves the translated content back to XML.

#     Args:
#         input_xml_path (str): Path to the input XML file.
#         metadata_path (str): Path to the metadata JSON file.
#         max_retries (int): Maximum number of retries for batch operations.
#         retry_delay (int): Delay in seconds between retries.

#     Returns:
#         bool: True if the process succeeds, False otherwise.
#     """
#     logger.info(
#         f"starting translation batch {journal_name}...",
#         extra={
#             "input_xml": input_xml_path,
#             "metadata_path": metadata_path,
#             "journal_name": journal_name
#         }
#     )
#     try: # data initialization:
#         # get metadata
#         section_metadata = deserialize_json(metadata_path)
#         section_title = section_metadata.section_title_en

#         # Extract page groups and split XML content
#         page_groups = extract_page_groups_from_metadata(section_metadata)
#         xml_content = get_text_from_file(input_xml_path)
#         sections = split_xml_pages(xml_content, page_groups)

#     except Exception as e:
#         # Log the error with full traceback
#         logger.error(
#             "Could not initialize data for translation batching {journal_name}", exc_info=True)
#         raise  # Re-raise the exception to escalate

#     # Create GPT messages for translation

#     user_message_wrapper = lambda section: f"Translate this section with title {section_title}:\n{section}"
#     messages = generate_messages(system_message_translate, user_message_wrapper, sections)

#     # Create JSONL file for batch processing
    
#     jsonl_file = create_jsonl_file_for_batch(messages, translate_batch_jsonl)
#     if not jsonl_file:
#         logger.error(
#             "Failed to create JSONL file for translation batch.",
#             exc_info=True  # Logs the exception traceback if one exists
#         )
#         raise RuntimeError("Failed to create JSONL file for translation batch.")
    
#     for attempt in range(max_retries): # batching logic requires multiple retries due to issues with API:
#         try:
#             # Start the batch
#             batch = start_batch(jsonl_file, description="Batch for translating journal")
#             batch_id = batch.get("id")
#             if not batch_id:
#                 raise RuntimeError("Batch started but no ID was returned.")
            
#             print(f"Batch for translation started successfully on attempt {attempt + 1}. ID: {batch_id}")

#             # Poll for batch completion
#             print("Polling for batch completion...")
#             results = poll_batch_for_response(batch_id)

#             if results:
#                 break # exit the retry loop
#             else:
#                 raise RuntimeError("Unknown error. No results from batch polling.")
            
#         except Exception as e:
#             logger.error(
#                 f"Attempt {attempt + 1} failed during translation for journal '{input_xml_path}'. Retrying in {retry_delay} seconds...",
#                 exc_info=True
#             )
#             time.sleep(retry_delay)
#     else:
#         logger.error(f"Failed to complete translation after {max_retries} retries for journal '{input_xml_path}'.")
#         raise RuntimeError("Unable to run translate batch.")
        
#     # Save translated content back to XML
#     try: 
#         print("Saving translated content back to XML...")
#         translated_sections = []
#         for i, translated_content in enumerate(results):
#             translated_sections.append(translated_content)

#         save_pages_to_xml(translated_sections, xml_output_path, overwrite=True)
#         print(f"Translated journal saved to {xml_output_path}")
#     except Exception as e:
#         raise RuntimeError("Failed to save translation data.")


In [ ]:
# testing
set_api_client()
msgs = generate_messages("you are assisting a software engineering/researcher looking to develop new AI platforms and processes.", lambda x: x, ["why is AI suddenly successful?", "What is the (immediate) future of AI?"])
run_immediate_chat_process(msgs[1])

In [ ]:
# testing
model_settings = {
    "gpt-4o": {
        "max_tokens": 3000,
        "context_limit": 20000,  # Total context limit for the model
        "temperature": 1.3
    }}

set_model_settings(model_settings)
batch_id = run_single_oa_batch(["what is the square root of 2?", "why is the sky blue?"], "you are are explaining complex ideas to a 9 year old child.")

poll_batch_for_response(batch_id, 10)

msgs = generate_messages("you are assisting a software engineering/researcher looking to develop new AI platforms and processes.", lambda x: x, ["why is AI suddenly successful?", "What is the (immediate) future of AI?"])
run_immediate_chat_process(msgs[1])

get_last_batch_response()